# Package installation and loading

In [2]:
!pip install simpletransformers
!pip install pyjarowinkler
!pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 258 kB 8.2 MB/s 
     |████████████████████████████████| 1.8 MB 24.2 MB/s 
     |████████████████████████████████| 103 kB 58.0 MB/s 


In [98]:
from enum import Enum
import logging
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel
import torch
import Levenshtein
from pyjarowinkler import distance as jw
import numpy as np
from itertools import cycle

# Supporting functions

In [4]:
def load_conllu_dataset(datafile):
    arr = []
    with open(datafile, encoding='utf-8') as inp:
        strings = inp.readlines()
    for s in strings:
      if (s[0] != "#" and s.strip()):
          split_string = s.split('\t')
          arr.append([split_string[1] + " " + split_string[3]+ " " + split_string[5], split_string[2]])    
    return pd.DataFrame(arr, columns=["input_text", "target_text"])

In [10]:
def dameraulevenshtein(s1, s2):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1

    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition
    return d[lenstr1-1,lenstr2-1] 

In [59]:
def prepare_metrics(given_list, metrics):
    raw_list = []    
    for i in given_list:
        raw_list.append(i[2])
    Q1, Q3 = np.percentile(raw_list, [25,75])
    IQR=Q3-Q1
    minimum=Q1-1.5*IQR
    maximum=Q3+1.5*IQR
    result_list = []
    errors_data = pd.DataFrame(columns=['TRUE', 'PRED', 'METRICS', 'RESULT'])
    counter = 0
    for i in given_list:
        if (i[2] > minimum and i[2] < maximum):
            result_list.append(i[2])
        elif (i[2] < minimum or i[2] > maximum):
            errors_data.loc[counter] = [i[0], i[1], metrics, i[2]]
            counter = counter + 1
    return raw_list, result_list, errors_data

# Setting global variables

In [8]:
modus = Enum("modus", "eval pred")

In [87]:
MODEL_NAME = "facebook/bart-large"
DATA_PRED_NAME = "/content/ab_test.conllu"
EVAL_NAME = "/content/dev_leg.conllu"
MODUS = modus.pred

In [88]:
eval_df = load_conllu_dataset(EVAL_NAME)
if (MODUS == modus.pred):
  pred_data = load_conllu_dataset(DATA_PRED_NAME)["input_text"].tolist()

 # Model loading

In [15]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 10,
    "train_batch_size": 8,
    "num_train_epochs": 2,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    # "silent": True,
    "evaluate_generated_text": True,
    "evaluate_during_training": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "save_best_model": False,
    "max_length": 15,
    "save_steps": -1,
    "evaluate_during_training_steps": 2000,
}

# working models: facebook/bart-large
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name=MODEL_NAME, 
    args=model_args,
    use_cuda = torch.cuda.is_available()
)

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/971M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

# Model evaluation

In [27]:
predictions = model.predict(eval_df["input_text"].tolist())

Generating outputs:   0%|          | 0/2061 [00:00<?, ?it/s]

In [29]:
predictions[1:10]

['Симонов',
 '. PUNCT _',
 'Се признал',
 'яз PRON Case=N',
 'пречис',
 'Богород',
 'Кузьми',
 'монастыр',
 'игумен N']

In [47]:
print('Accuracy score: ' + str(sum([1 if label == pred else 0 for pred, label in zip(predictions, eval_df['target_text'].tolist())])/len(predictions)) + '%')

Accuracy score: 0.012192163047434187%


In [73]:
error_datasets = []

In [74]:
levenshteins = [(label, pred, Levenshtein.distance(label, pred)) for pred, label in zip(predictions, eval_df['target_text'].tolist())]       

In [75]:
raw_levenshteins, cleared_levenshteins, errors_levenshteins = prepare_metrics(levenshteins, 'Levenshtein distance')
print('Raw average Levenshtein distance: ' + str(sum(raw_levenshteins)/len(raw_levenshteins)))
print('Normalized average Levenshtein distance: ' + str(sum(cleared_levenshteins)/len(cleared_levenshteins)))
error_datasets.append(errors_levenshteins) 

Raw average Levenshtein distance: 6.504124711876744
Normalized average Levenshtein distance: 6.243870647383775


In [76]:
damerau_levenshteins = [(label, pred, dameraulevenshtein(label, pred)) for pred, label in zip(predictions, eval_df['target_text'].tolist())]

In [77]:
raw_damerau_levenshteins, cleared_damerau_levenshteins, errors_damerau_levenshteins = prepare_metrics(damerau_levenshteins, 'Damerau-Levenshtein distance')
print('Raw average Damerau-Levenshtein distance: ' + str(sum(raw_damerau_levenshteins)/len(raw_damerau_levenshteins)))
print('Normalized average Damerau-Levenshtein distance: ' + str(sum(cleared_damerau_levenshteins)/len(cleared_damerau_levenshteins)))
error_datasets.append(errors_damerau_levenshteins)

Raw average Damerau-Levenshtein distance: 6.503821424238748
Normalized average Damerau-Levenshtein distance: 6.243560300415865


In [78]:
jaro_winklers = [(label, pred, jw.get_jaro_distance(label, pred)) for pred, label in zip(predictions, eval_df['target_text'].tolist())]

In [79]:
raw_jaro_winklers, cleared_jaro_winklers, errors_jaro_winklers = prepare_metrics(jaro_winklers, 'Jaro-Winkler distance')
print('Raw average Jaro-Winkler distance: ' + str(sum(raw_jaro_winklers)/len(raw_jaro_winklers)))
print('Normalized average Jaro-Winkler distance: ' + str(sum(cleared_jaro_winklers)/len(cleared_jaro_winklers)))
error_datasets.append(errors_jaro_winklers)

Raw average Jaro-Winkler distance: 0.7222146063326137
Normalized average Jaro-Winkler distance: 0.8144487161883487


In [80]:
errors = pd.concat(error_datasets)

In [81]:
errors.to_csv('errors_' + (MODEL_NAME.split('/')[len(MODEL_NAME.split('/')) - 1] if '/' in MODEL_NAME else MODEL_NAME) + '.csv', index = False, encoding='utf-8')

# Prediction

In [99]:
predictions = cycle(model.predict(pred_data))

Generating outputs:   0%|          | 0/11 [00:00<?, ?it/s]

In [107]:
with open(DATA_PRED_NAME, encoding='utf-8') as inp:
    strings = inp.readlines()
    predicted = []
    for s in strings:
      if (s[0] != "#" and s.strip()):
          split_string = s.split('\t')
          split_string[2] = next(predictions)
          joined_string = '\t'.join(split_string)
          predicted.append(joined_string)
          continue
      predicted.append(s)      
    with open("predictions.conllu", 'w', encoding='utf-8') as out:
      out.write('\n'.join(predicted))